In [37]:
import tensorflow as tf
import numpy as np

### 텐서 = 배열(Array)
* Rank

몇 차원 배열인가를 의미
|Rank   |Math Entity                        |Example                                                    |
|-------|-------------------------------    |----------                                                 |
|0      |Scalar(magnitude only)             |s = 483                                                    |
|1      |Vector(magnitude and direction)    |v = [1.1, 2.2, 3.3]                                        |
|2      |Maxrix(table of numbers)           |m = [[1, 2, 3], [1, 2, 3], [1, 2, 3]]                      |
|3      |3-Tensor(cube of numbers)          |c = [[[1], [2], [3]], [[4], [5], [6]], [[7], [8], [9]]]    |
|n      |n-Tensor(n 차원 배열)              |...                                                        |

스칼라(Scalar)는 0차원 배열
rank가 n이면 n차원 배열

* 텐서의 Shape

각 축이 몇 개의 엘리먼트(Element)들로 구성되었는지 나타내는 값.
shape = (2, 3) 2행 3열

* 텐서의 Type

텐서가 담을 수 있는 데이터의 타입을 의미. tf.float32, tf.int32 등


In [38]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]]) #행렬 텐서

In [39]:
s = tf.constant(42) #스칼라 텐서

In [40]:
t.shape, t.dtype

(TensorShape([2, 3]), tf.float32)

In [41]:
print(t[:, 1:])
t[..., 1, tf.newaxis]

tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)


<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [42]:
print(t + 10)
print(tf.square(t))
print(tf.transpose(t))
t @ tf.transpose(t) # @: matmul

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 4.]
 [2. 5.]
 [3. 6.]], shape=(3, 2), dtype=float32)


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

12. 2 텐서와 넘파이
numpy는 기본적으로 64비트 정밀도 사용
tensorflow는 32비트 사용
왜냐하면 신경망은 32비트로도 충분, 더 빠름, 메모리도 적게 사용
넘파이 배열로 텐서를 만들 때 dtype=tf.float32로 지정해야함

In [44]:
a = np.array([2., 4., 5.])
print(tf.constant(a)) #numpy 행렬로 텐서 생성
print(t.numpy()) # 또는 np.array(t)
print(tf.square(a))
print(np.square(t))

tf.Tensor([2. 4. 5.], shape=(3,), dtype=float64)
[[1. 2. 3.]
 [4. 5. 6.]]
tf.Tensor([ 4. 16. 25.], shape=(3,), dtype=float64)
[[ 1.  4.  9.]
 [16. 25. 36.]]


12. 3 타입 변환
타입변환은 성능을 크게 감소시킬 수 있음
자동 타입 변환을 방지하기 위해 tensorflow는 자동으로 타입 변환시키지 않음
호환되지 않는 타입의 텐서로 연산 실행하면 예외 발생
"정수 + 실수 텐서"x, "32비트 실수" + "64비트 실수"x

In [46]:
# print(tf.constant(2.) + tf.constant(40)) 오류 발생
# print(tf.constant(2.) + tf.constant(40., dtype=tf.float64))

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2]

12. 4 변수
위의 Tensor는 변경이 불가능한 객체. 즉 텐서의 내용을 변경할 수 없음
따라서 일반적인 텐서로는 역전파로 변경되어야하는 신경망의 **가중치**를 구현 불가능
또한 시간에 따라 변경되어야 할 파라미터도 있음(e.g. 모멘텀 옵티마이저는 과거 gradient를 계속 업데이트)
이떄 **tf.Variable** 사용

In [54]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

tf.Variable은 tf.Tensor와 비슷하게 동작함
다른 점은 **assign()** 메서드를 사용하여 개별 원소 변경 가능
**scatter_update(), scatter_nd_update()** 메서드를 사용하여 개별 원소(또는 슬라이스)를 수정 가능

In [55]:
print(v.assign(2 * v)) # => [[2., 4., 6.], [8., 10., 12]]
print(v[0, 1].assign(42)) # [0, 1] 변수를 42로 변경
print(v[:, 2].assign([0., 1.])) # 2열 각각을 0.과 1.으로 변경
v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.])
# [0, 0] 원소와 [1, 2] 원소를 각각 100.과 200.로 변경

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>
<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>
<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>


<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

12. 2.5 다른 데이터 구조
* 희소 텐서(sparse tensor, tf.SparseTensor)
tf.sparse패키지는 희소 텐서를 위한 연산 제공
* 텐서 배열(tensor array, tf.TensorArray)
텐서의 리스트. 길이 동적 변경 가능. 
리스트 내의 모든 텐서는 동일한 크기와 타입이어야 함
* 래그드 텐서(ragged tensor, tf.RaggedTensor)
리스트의 리스트를 나타냄. 동일한 데이터 타입, 길이는 다를 수 있음
* 문자열 텐서(string tensor)
tf.string 타입의 텐서. 
* 집합(set)
일반적인 텐서로 나타냄. tf.constant()
* 큐(queue)

12. 3 사용자 정의 모델과 훈련 알고리즘

12. 3.1 사용자 정의 손실함수
regression model을 훈련하는 데 훈련 세트에 잡음 데이터가 조금 있다고 가정하면 huber 사용(keras.losses.Huber 도 있음)

In [57]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) /2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

12. 3.2 사용자 정의 요소를 가진 모델을 저장 & 로드하기


tf.Tensor(0.0050000004, shape=(), dtype=float32)
